This is the training of the BERT models. It returns a list of probabilities for each review used as input for the classification model

In [ ]:
!pip install pandas
!pip install simpletransformers

In [ ]:
data = pd.read_csv('review_helpful.csv', sep = ";")
data['review'] = data['review'].fillna('')

In [ ]:
data['review'] = data['review'].str.lower()

In [ ]:
(data['helpful'] != 0).astype(int)

In [ ]:
data = pd.DataFrame({'text': data['review'], 'labels': data['helpful']})

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(data, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=42)

In [ ]:
from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import ClassificationArgs
model_args = ClassificationArgs(
    use_multiprocessing=False,
    use_multiprocessing_for_evaluation=False,
    max_seq_length=128,
    eval_batch_size=256
)
# Create a TransformerModel

#model = ClassificationModel('bert', 'bert-base-uncased', args=model_args, use_cuda=True)
#model = ClassificationModel('roberta', 'roberta-large', args=model_args, use_cuda = True)
model = ClassificationModel('roberta', 'roberta-base', args=model_args, use_cuda=True)
#model = ClassificationModel('bert', 'bert-large-uncased', args=model_args, use_cuda=True)
#model = ClassificationModel('distilbert', 'distilbert-base-uncased', args=model_args, use_cuda=True)
#model = ClassificationModel('albert', 'albert-base-v2', args=model_args, use_cuda=True)

In [ ]:
import os
#os.environ["OMP_NUM_THREADS"] = "1"

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
history = model.train_model(train_df, args={'num_train_epochs': 2, 'learning_rate': 3e-5, 'overwrite_output_dir': True,  'train_batch_size' : 256})

In [ ]:
result_train, model_outputs_train, wrong_predictions_train = model.eval_model(train_df)
result_val, model_outputs_val, wrong_predictions_val = model.eval_model(val_df)

# Save model predictions

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(data)

In [ ]:
model_outputs_data = pd.concat([data, pd.DataFrame(model_outputs)], axis = 1)

In [ ]:
model_outputs_data.to_csv('model_outputs_data.csv', sep = ";")

# Get model performance

In [ ]:
#results = pd.DataFrame(result_val, index = [0])
results['train_loss'] = result_train['eval_loss']
results['name'] = 'bert-base-uncased'
results['train_accuracy'] = result_train['accuracy']
#results['seq_length'] = 64

In [ ]:
results_new = pd.DataFrame(result_val, index = [0])
results_new['train_loss'] = result_train['eval_loss']
results_new['train_accuracy'] = result_train['accuracy']
results_new['name'] = 'roberta-base'
#results_new['seq_length'] = 512
results = pd.concat([results, results_new])

In [ ]:
results.reset_index(drop = True, inplace = True)

In [ ]:
results.to_csv('roberta_seq_length.csv', sep = ";")

In [ ]:
model.save_model("./roberto")